In [1]:
from io import BytesIO
from urllib.request import urlopen
import librosa
from transformers import Qwen2AudioForConditionalGeneration, Qwen2AudioProcessor
import torch

In [2]:

processor = Qwen2AudioProcessor.from_pretrained("/disk0/aitogether/ai-english/yi_model/model/qwen/qwen2-audio-7b-instruct")
model = Qwen2AudioForConditionalGeneration.from_pretrained("/disk0/aitogether/ai-english/yi_model/model/qwen/qwen2-audio-7b-instruct",torch_dtype = "auto", device_map="auto").eval()


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
def analyse_audio(query,audio):
    conversation = [
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {"role": "user", "content": [
            {"type": "audio", "audio": "/disk0/aitogether/xishaojian/audio_lab/audio_resources/"+audio},
            {"type":"text","text":query}
        ]},
    ]
    text = processor.apply_chat_template(conversation,chat_template = processor.default_chat_template, add_generation_prompt=True, tokenize=False)
    audios = []
    for message in conversation:
        if isinstance(message["content"], list):
            for ele in message["content"]:
                if ele["type"] == "audio":
                    if "audio_url" in ele:
                        audios.append(
                            librosa.load(ele['audio_url'], sr=processor.feature_extractor.sampling_rate)[0]
                        )
                    elif "audio" in ele:
                        audios.append(
                            librosa.load(ele['audio'], sr=processor.feature_extractor.sampling_rate)[0]
                        )
    inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True)
    inputs["input_ids"] = inputs.input_ids.to("cuda")
    generate_ids = model.generate(**inputs, max_length=4096)
    generate_ids = generate_ids[:, inputs.input_ids.size(1):]
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print(response)

In [6]:
analyse_audio("里面有多位说话者，请提出每个说话者对应的内容","PPT+Teacher+Student.MP3")

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


第一位说话者提到中国目前垃圾填埋和焚烧的问题，以及垃圾分类没有解决的挑战。第二位说话者是垃圾填埋组的代表，他/她强调垃圾填埋会导致周围土地受到污染。
